In [200]:
import nltk
import re
import pickle
import pymystem3
m = pymystem3.Mystem() #для использования лемматизации
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import gensim
from nltk.corpus import stopwords
stop_words = stopwords.words("russian")

from string import punctuation
import math
from tqdm import tqdm_notebook

import csv

In [62]:
def unpack(data):
    input = open(data, 'rb')
    obj = pickle.load(input)
    input.close()
    return obj

# частоты лем униграмм
unigrams = unpack('1stemgrams.data')
# обрезали по частоте 20, а еще убрали пробел из начала слов
unigrams = {w[1:]:f for w,f in unigrams.items()}

In [64]:
# читает из файла, убирает двойные пробелы и ручные переносы, последний \n
def reading(file):
    f = open('texts\\{}.txt'.format(file), 'r', encoding='utf-8')
    text = f.read()
    text = text.replace('  ', ' ')
    text = text.replace('-\n', '')
    if text[-1] == '\n': # убираем последний \n, если такой есть 
        text = text[:-1]
    f.close()
    return text

In [112]:
def minimummaker(): #ф-ция превращения текстового файла с минимумом в питоновский список. Запаковка списка
    with open('min.txt', encoding='utf-8') as file:
        lemtokens = [morph.parse(i)[0].normal_form for i in re.findall('\w+', file.read().lower())] #делаем список лемм
        minimum = list(set(lemtokens)) #убираем повторы
        output = open('minimum.pkl', 'wb')
        pickle.dump(minimum, output, 2)
        output.close()

def loadminimum(): #распаковка cписка с минимумом. Возвращает неупорядоченный список
    input = open('minimum.pkl', 'rb')
    minimum = pickle.load(input)
    input.close()
    return minimum

#minimummaker()
minimum = loadminimum()
#print(minimum[-50:]) #проверка работы
#print(len(minimum)) #2549 слов в списке

['живой', 'потребовать', 'математический', 'старший', 'водить', 'занятие', 'восемьсот', 'исправлять', 'марин', 'сантиметр', 'форма', 'столько', 'туча', 'узнать', 'длина', 'пиво', 'улыбнуться', 'география', 'родиться', 'глубокий', 'закумокнуть', 'рубль', 'заниматься', 'строго', 'компьютер', 'кончить', 'четверть', 'сон', 'много', 'доклад', 'урожай', 'район', 'но', 'попытаться', 'зависеть', 'сердце', 'труд', 'тренер', 'тайна', 'исполняться', 'итак', 'тысяча', 'ученик', 'мила', 'диск', 'ужин', 'некуда', 'захотеть', 'однажды', 'увлечься']
2549


In [235]:
# загрузка модели
def model_loading(file):
    model = gensim.models.KeyedVectors.load_word2vec_format(file, binary=False)
    model.init_sims(replace=True)
    print('Done!') 
    return model

model = model_loading('news_upos_cbow_600_2_2018.vec') #на загрузку тратится минут 10

KeyboardInterrupt: 

In [131]:
# загрузка словаря ASIS
with open('syns.data', 'rb') as f:
     asis = pickle.load(f)

In [183]:
class Token():
    def __init__(self, w):
        if isinstance(w, dict): # если получили результат работы mystem
            self.text = w['text']  # сам токен
            
            # определяет, сделан ли анализ и, соответственно, рассматривать ли как слово, требующее упрощения
            gram = w.get('analysis')
            if gram:                
                if not named_entity(gram[0]):  # именованная сущность или нет
                    self.lexem = gram[0]['lex']  # лемма                    
                else:
                    self.lexem = '_NAMED_ENTITY_'
                
                self.pos = pos(gram[0]['gr'])  # часть речи
                    
            elif any(p in w['text'] for p in punctuation+'–«»'): # если это знак пунктуации (может быть с пробелом!)
                self.lexem = '_PUNKTUATION_'
                self.pos = None
            
            elif not re.findall('\S',w['text']): # если это только пробельные символы
                self.lexem = '_SPACE_'
                self.pos = None
                
            # остальное - неизвестная и ненужная ерунда?
            else:
                self.lexem = '_UNK_'
                self.pos = None
            
            self.len = len(w['text'])
            
        elif isinstance(w, Token): # для определения объектов дочернего класса - проверяем что уже является объектом Родительского
            self.text = w.text
            self.num = w.num
            self.lexem = w.lexem 
            self.len = w.len
            self.pos = w.pos
            self.complexity = w.complexity
        
        elif isinstance(w, str): # если хотим как класс токен определить строку
            self.text = w # задаём саму строку текстом токена
            self.lexem, self.pos = tagging(w) # определяем тег и лемму с помощью пайморфи
            self.len = len(self.lexem)
        
        self.num = None # номер в тексте
        self.complexity = None # сложность слова
        
    def complexity_params(self, param = 'freq'):
        # если по частотности
        if param == 'freq':
            self.complexity = unigrams.get(self.lem, 0)
            
        # если по коэффициенту информативности. Отрицательное значение. Чем он меньше, тем сложнее
        elif param == 'inf':
            self.complexity = math.log((unigrams.get(self.lexem, 0)+1)/(sum(f for f in unigrams.values())+1))
        
    
    def is_complex(self, threshold = '600', use_min = False, len_threshold = 1000):
        exceptions = ['_NAMED_ENTITY_', '_PUNKTUATION_', '_SPACE_', '_UNK_']
        # проверка, что это слово и что его нужно рассматривать как сложное (не нарицательное)
        if not any(exception in self.lexem for exception in exceptions):
            
            # если показатель сложности - вхождение в минимум
            if use_min:
                if self.lexem not in minimum:
                    return True
                else:
                    return False

            # если показатель сложности - пороговое значение сложности
            # также может использоваться длина. По умолчанию слишком большое - 1000 (т.е. этот параметр не учитывается)
            else:
                if self.complexity < threshold or self.len > len_threshold:
                    return True
                else:
                    return False
        else:
            return False
        
'''Пример вывода атрибутов объектов класса 
token.num, token.text, token.lem, token.pos, token.complexity, token.is_complex(threshold, use_min):

223 Кукушкин _NAMED_ENTITY_ S -19.06687873346149 False
224   _SPACE_ None -19.06687873346149 False
225 является являться V -7.502177090710664 False
226   _SPACE_ None -19.06687873346149 False
227 должником должник S -11.184186527172464 True
228   _SPACE_ None -19.06687873346149 False
229 банка банк S -9.300701131889967 False
230 .  _PUNKTUATION_ None -19.06687873346149 False
'''

'Пример вывода атрибутов объектов класса \ntoken.num, token.text, token.lem, token.pos, token.complexity, token.is_complex(threshold, use_min):\n\n223 Кукушкин _NAMED_ENTITY_ S -19.06687873346149 False\n224   _SPACE_ None -19.06687873346149 False\n225 является являться V -7.502177090710664 False\n226   _SPACE_ None -19.06687873346149 False\n227 должником должник S -11.184186527172464 True\n228   _SPACE_ None -19.06687873346149 False\n229 банка банк S -9.300701131889967 False\n230 .  _PUNKTUATION_ None -19.06687873346149 False\n'

In [142]:
# вытаскивает часть речи из разбора майстем
def pos(gram):
    if ',' in gram:
        gram = gram.split(',')[0]
    if '=' in gram:
        gram = gram.split('=')[0]
    return gram

# определяет по тегам, является ли именованной сущностью
def named_entity(gram):
    markers = {'сокр': ' - сокращение', 'фам': ' - фамилия', 'имя': ' - имя собственное', 'гео': ' - название места', }
    if any(m in gram['gr'] for m in markers.keys()):
        return True
    else:
        return False

# для совсем новых слов: определяем тег пайморфи, переводим в формат майстем - так быстрее, чем майстемом
def tagging(w):
    # словарь, переводящий теги пайморфи в теги майстема
    tags = {'ADJF':'ADJ',
    'ADJS' : 'ADJ',
    'ADVB' : 'ADV',
    'COMP' : 'ADV',
    'GRND' : 'VERB',
    'INFN' : 'VERB',
    'NOUN' : 'NOUN',
    'PRED' : 'ADV',
    'PRTF' : 'ADJ',
    'PRTS' : 'VERB',
    'VERB' : 'VERB'}
    tag = morph.parse(w)[0].tag.POS
    if tag in tags:
        return morph.parse(w)[0].normal_form, tags[tag]
    else:
        return '_UNK_', None

#анализ текста 
def text_structuring(text, param, threshold, use_min):
    # анализирует текст 
    analysis = m.analyze(text)
    tokens = []
    for i, w in enumerate(analysis): # состаляем список объектов Tokens
        token = Token(w)
        token.num = i # добавляем токену в атрибуты его номер в тексте
        token.complexity_params(param) # переопределяем сложность на основе выбранного параметра
        print(token.num, token.text, token.lexem, token.pos, token.complexity, token.is_complex(threshold, use_min))
        tokens.append(token)
    return tokens

In [143]:
# ПАРАМЕТРЫ анализа слов
complexity_type = 'inf'
global_threshold = -8.5
use_min=True

In [189]:
isinstance(tokens[-1], Token)

True

In [188]:
text = reading('news1')
tokens = text_structuring(text, complexity_type, global_threshold, use_min)

0 Экс экс S -12.407584813777852 True
1 - _PUNKTUATION_ None -19.06687873346149 False
2 депутат депутат S -9.544944468987213 False
3   _SPACE_ None -19.06687873346149 False
4 из из PR -5.430146815838169 False
5   _SPACE_ None -19.06687873346149 False
6 Астрахани _NAMED_ENTITY_ S -19.06687873346149 False
7   _SPACE_ None -19.06687873346149 False
8 обманул обманывать V -9.722269740680273 True
9   _SPACE_ None -19.06687873346149 False
10 ВТБ _NAMED_ENTITY_ S -19.06687873346149 False
11   _SPACE_ None -19.06687873346149 False
12 на на PR -4.174175661002731 False
13   _SPACE_ None -19.06687873346149 False
14 750 _UNK_ None -19.06687873346149 False
15   _SPACE_ None -19.06687873346149 False
16 млн _UNK_ None -19.06687873346149 False
17 ,  _PUNKTUATION_ None -19.06687873346149 False
18 переписав переписывать V -11.046279583564518 True
19   _SPACE_ None -19.06687873346149 False
20 имущество имущество S -9.643525957370555 True
21 
 _SPACE_ None -19.06687873346149 False
22 
 _SPACE_ None -19.0668

179 рублей рубль S -8.4421182633064 False
180 .  _PUNKTUATION_ None -19.06687873346149 False
181 В в PR -3.486155131943022 False
182   _SPACE_ None -19.06687873346149 False
183 МВД _NAMED_ENTITY_ S -19.06687873346149 False
184   _SPACE_ None -19.06687873346149 False
185 России _NAMED_ENTITY_ S -19.06687873346149 False
186   _SPACE_ None -19.06687873346149 False
187 по по PR -5.1820044394556914 False
188   _SPACE_ None -19.06687873346149 False
189 Астраханской астраханский A -11.87544940342511 True
190   _SPACE_ None -19.06687873346149 False
191 области область S -8.188095962474803 False
192   _SPACE_ None -19.06687873346149 False
193 возбудили возбуждать V -9.872871016106204 True
194   _SPACE_ None -19.06687873346149 False
195 уголовное уголовный A -9.858339983431934 True
196   _SPACE_ None -19.06687873346149 False
197 дело дело S -6.454827778200301 False
198   _SPACE_ None -19.06687873346149 False
199 по по PR -5.1820044394556914 False
200   _SPACE_ None -19.06687873346149 False
201 ф

In [ ]:
for token in tokens:
    if token.is_complex(global_threshold, use_min):
        comp_token = Complex_word(token)
        comp_token.search_substituts(base_type='model')
        if comp_token.substituts:
            syns_lexems = [syn.lexem for syn in comp_token.substituts]
            print(comp_token.lexem, 'model:', syns_lexems)
        '''comp_token.search_substituts(base_type='yarn')
        if comp_token.substituts:
            syns_lexems = [syn.lexem for syn in comp_token.substituts]
            print(comp_token.lexem, 'yarn:', syns_lexems)
        comp_token.search_substituts(base_type='asis')
        if comp_token.substituts:
            syns_lexems = [syn.lexem for syn in comp_token.substituts]
            print(comp_token.lexem, 'asis:', syns_lexems)'''

In [ ]:
class Complex_word(Token):
    def __init__(self, w):
        super().__init__(w)
        self.substituts = None
    # список замен, в зависимости от выбранной базы
    def search_substituts(self, base_type='model'):
        
        # поиск по модели
        def model_search(lexem, pos):
            query = str(lexem+'_'+pos)
            #print(query)
            if query in model:
                # формируем список квазисинонимов той же части речи
                return [Token(syn) for syn in model.most_similar(positive=[query])]
            else:
                return [] 

        # поиск по YARN
        def yarn_search(target, filepath = 'yarn-synsets1.csv'):
            with open(filepath, "r", newline="") as file: # постепенный просмотр файла с синсетами (множествами синонимов)
                reader = csv.DictReader(file, delimiter=';')
                lst = []
                for i,row in enumerate(reader):
                    cur_line = row['words'].split(';') # считываем колонку с синсетами
                    if len(cur_line)>1:
                        if target in cur_line:
                            del(reader)
                            lst = [Token(c) for c in cur_line if ' ' not in c and c!=target] # формируем список однословных синонимов
                            #TODO: выделить неоднословные в отдельный класс и поискать их частотность по n-граммам?
                            break
            #print(lst)
            return lst 
        
        #поиск по ASIS
        def asis_search(target):
            if target in asis:
                return [Token(s) for s in asis[target] if ' ' not in s]
            else:
                return [] # формируем список однословных квазисинонимов

        
        if base_type == 'model':
            self.substituts = model_search(self.lexem, self.pos)
            
        if base_type == 'yarn':
            self.substituts = yarn_search(self.lexem)
        
        if base_type == 'asis':
            self.substituts = asis_search(self.lexem)
        
        return self

In [ ]:
for token in tokens:
    if token.is_complex(global_threshold, use_min):
        subst_search()